In [3]:
import pandas as pd

# 1. Load the Data
df = pd.read_csv('summerOly_medal_counts.csv')

# 2. Data Cleaning & Standardization (Crucial for the Paper)
# The paper specifies merging specific historical entities.
# We map Soviet Union/ROC to Russia, and East/West Germany to Germany.
country_mapping = {
    'Soviet Union': 'Russia',
    'ROC': 'Russia',           # Russian Olympic Committee (2020)
    'Unified Team': 'Russia',  # 1992 Transitional Team
    'East Germany': 'Germany',
    'West Germany': 'Germany'
}

# Create a clean copy and apply the mapping
df_clean = df.copy()
df_clean['NOC'] = df_clean['NOC'].replace(country_mapping)

# 3. Remove 1906 Intercalated Games (if present) as per paper rules
df_clean = df_clean[df_clean['Year'] != 1906]

# 4. Identify "Regulars"
# Definition: Any country that won at least 1 medal in the last two cycles (2020 or 2024).
recent_cycles = [2020, 2024]
recent_data = df_clean[df_clean['Year'].isin(recent_cycles)]

# Group by Country (NOC) and see if they have medals
# (Note: Your dataset only contains winners, so anyone in the 2020/2024 rows is a Regular)
regular_countries_list = recent_data['NOC'].unique()

print(f" identified {len(regular_countries_list)} 'Regular' countries (Active Winners).")

# 5. Split the Dataset
# Bucket A: Regulars (Historical data for current winners)
df_regulars = df_clean[df_clean['NOC'].isin(regular_countries_list)]

# Bucket B: Underdogs (Historical data for countries that didn't win recently)
# This includes defunct nations (e.g., Yugoslavia) or countries currently in a drought.
df_underdogs = df_clean[~df_clean['NOC'].isin(regular_countries_list)]

# 6. Save the Output
df_regulars.to_csv('data_bucket_A_regulars.csv', index=False)
df_underdogs.to_csv('data_bucket_B_underdogs.csv', index=False)

print("Split complete. Created 'data_bucket_A_regulars.csv' and 'data_bucket_B_underdogs.csv'.")

 identified 109 'Regular' countries (Active Winners).
Split complete. Created 'data_bucket_A_regulars.csv' and 'data_bucket_B_underdogs.csv'.


In [7]:
import pandas as pd
import numpy as np

# --- 1. LOAD DATA WITH CORRECT ENCODING ---
# We use encoding='cp1252' to handle special characters like "•"
df_medals = pd.read_csv('summerOly_medal_counts.csv', encoding='cp1252')
df_programs = pd.read_csv('summerOly_programs.csv', encoding='cp1252')
df_athletes = pd.read_csv('summerOly_athletes.csv', encoding='cp1252')

# If 'cp1252' still fails, try 'ISO-8859-1'
# df_programs = pd.read_csv('summerOly_programs.csv', encoding='ISO-8859-1')

print("Files loaded successfully!")

# --- CONTINUE WITH THE REST OF THE SCRIPT BELOW ---
# ... (Paste the rest of the script from the previous step here)

Files loaded successfully!


In [9]:
import pandas as pd
import numpy as np

# --- 1. LOAD DATA WITH ENCODING FIX ---
# usage of encoding='cp1252' fixes the "byte 0x95" error
df_medals = pd.read_csv('summerOly_medal_counts.csv', encoding='cp1252')
df_programs = pd.read_csv('summerOly_programs.csv', encoding='cp1252')
df_athletes = pd.read_csv('summerOly_athletes.csv', encoding='cp1252')

print("Files loaded successfully.")

# --- 2. STANDARDIZE COUNTRY NAMES ---
# Map historical countries to modern equivalents so they match across files
country_map = {
    'Soviet Union': 'Russia', 'URS': 'Russia', 'ROC': 'Russia', 'EUN': 'Russia',
    'East Germany': 'Germany', 'GDR': 'Germany',
    'West Germany': 'Germany', 'FRG': 'Germany',
    'United States': 'USA', 'Great Britain': 'GBR',
    'China': 'CHN', 'Japan': 'JPN', 'France': 'FRA' 
    # Add more if you see mismatches in your specific data
}

def clean_country(df, col_name):
    # Ensure column is string type before replacing
    df[col_name] = df[col_name].astype(str).replace(country_map)
    return df

df_medals = clean_country(df_medals, 'NOC')
df_athletes = clean_country(df_athletes, 'NOC')

# --- 3. CALCULATE "TOTAL EVENTS" (Denominator for ER) ---
# Clean the year columns (remove '*') and sum the event counts
year_cols = [c for c in df_programs.columns if c.replace('*','').strip().isdigit()]
total_events_map = {}

for year in year_cols:
    clean_year = int(year.replace('*','').strip())
    # Convert '•' or other non-numbers to 0, then sum
    col_data = pd.to_numeric(df_programs[year], errors='coerce').fillna(0)
    total_events_map[clean_year] = col_data.sum()

# --- 4. CALCULATE "COUNTRY PARTICIPATION" (Numerator for ER & NA) ---
# Group athletes by Year + Country to get counts
country_stats = df_athletes.groupby(['Year', 'NOC']).agg({
    'Event': 'nunique',  # Count unique events entered
    'Name': 'nunique'    # Count unique athletes (NA)
}).reset_index().rename(columns={'Event': 'Events_Participated', 'Name': 'NA'})

# --- 5. MERGE EVERYTHING INTO THE MASTER TABLE ---
# Start with medal winners
df_master = df_medals.merge(country_stats, on=['Year', 'NOC'], how='left')

# Fill NaNs (if a country won medals but has no athlete data, assume 0 or missing)
df_master['Events_Participated'] = df_master['Events_Participated'].fillna(0)
df_master['NA'] = df_master['NA'].fillna(0)

# Calculate ER (Participation Rate)
def calc_er(row):
    y = row['Year']
    # Avoid division by zero
    if y in total_events_map and total_events_map[y] > 0:
        return row['Events_Participated'] / total_events_map[y]
    return 0

df_master['ER'] = df_master.apply(calc_er, axis=1)

# --- 6. CALCULATE HOST EFFECT (HE) ---
# Map Host City to Country Code (Update this list based on your data's NOC codes)
hosts = {
    2028: 'USA', 2024: 'FRA', 2020: 'JPN', 2016: 'BRA', 
    2012: 'GBR', 2008: 'CHN', 2004: 'GRE', 2000: 'AUS',
    1996: 'USA', 1992: 'ESP', 1988: 'KOR', 1984: 'USA', 
    1980: 'Russia', 1976: 'CAN', 1972: 'Germany'
}

df_master['HE'] = df_master.apply(lambda x: 1 if hosts.get(x['Year']) == x['NOC'] else 0, axis=1)

# --- 7. CALCULATE ATHLETIC PERFORMANCE (AP) ---
# Weighted sum of medals from the LAST 2 Olympics
df_master['Score'] = (df_master['Gold']*3) + (df_master['Silver']*2) + (df_master['Bronze']*1)

# Sort by Country then Year to calculate lag
df_master = df_master.sort_values(['NOC', 'Year'])

# Shift 1 (Previous Games) and Shift 2 (Games before that)
df_master['Score_Lag1'] = df_master.groupby('NOC')['Score'].shift(1).fillna(0)
df_master['Score_Lag2'] = df_master.groupby('NOC')['Score'].shift(2).fillna(0)

# AP = Sum of scores from last two games
df_master['AP'] = df_master['Score_Lag1'] + df_master['Score_Lag2']

# --- 8. MUNDLAK VARIABLES (Long-Term Averages) ---
# Calculate the historical average for each country
mundlak_ER = df_master.groupby('NOC')['ER'].mean().rename('Mundlak_ER')
mundlak_NM = df_master.groupby('NOC')['Score'].mean().rename('Mundlak_NM')

# Merge these back into the main table
df_master = df_master.merge(mundlak_ER, on='NOC').merge(mundlak_NM, on='NOC')

# --- 9. FINAL FILTER FOR "REGULARS" ---
# Filter for countries that won at least 1 medal in 2020 or 2024
recent_winners = df_master[df_master['Year'].isin([2020, 2024])]['NOC'].unique()
df_regulars = df_master[df_master['NOC'].isin(recent_winners)].copy()

# Add Constant column (needed for the math model later)
df_regulars['Const'] = 1

# Save to CSV
df_regulars.to_csv('final_model_data.csv', index=False)
print(f"Success! Processed {len(df_regulars)} rows. Saved to 'final_model_data.csv'.")

Files loaded successfully.
Success! Processed 1242 rows. Saved to 'final_model_data.csv'.


In [10]:
import pandas as pd
import numpy as np

# --- 1. LOAD DATA (With Fix for Special Characters) ---
# The 'cp1252' encoding fixes the crash caused by bullet points in the CSV
df_medals = pd.read_csv('summerOly_medal_counts.csv', encoding='cp1252')
df_programs = pd.read_csv('summerOly_programs.csv', encoding='cp1252')
df_athletes = pd.read_csv('summerOly_athletes.csv', encoding='cp1252')

# --- 2. STANDARDIZE COUNTRY NAMES ---
# Ensure countries match across all three files
country_map = {
    'Soviet Union': 'Russia', 'URS': 'Russia', 'ROC': 'Russia', 'EUN': 'Russia',
    'East Germany': 'Germany', 'GDR': 'Germany', 'West Germany': 'Germany', 'FRG': 'Germany',
    'United States': 'USA', 'Great Britain': 'GBR', 'China': 'CHN', 'Japan': 'JPN', 'France': 'FRA'
}

def clean_country(df, col_name):
    df[col_name] = df[col_name].astype(str).replace(country_map)
    return df

df_medals = clean_country(df_medals, 'NOC')
df_athletes = clean_country(df_athletes, 'NOC')

# --- 3. CALCULATE "TOTAL EVENTS" (Denominator for ER) ---
# Clean columns and sum up total events available per year
year_cols = [c for c in df_programs.columns if c.replace('*','').strip().isdigit()]
total_events_map = {}
for year in year_cols:
    clean_year = int(year.replace('*','').strip())
    # Force non-numeric characters (like •) to NaN, then 0
    col_data = pd.to_numeric(df_programs[year], errors='coerce').fillna(0)
    total_events_map[clean_year] = col_data.sum()

# --- 4. CALCULATE "COUNTRY PARTICIPATION" (Numerator) ---
# This gives us NA (Number of Athletes) which is KEY for Underdogs
country_stats = df_athletes.groupby(['Year', 'NOC']).agg({
    'Event': 'nunique', 
    'Name': 'nunique'
}).reset_index().rename(columns={'Event': 'Events_Participated', 'Name': 'NA'})

# --- 5. MERGE & CALCULATE FEATURES ---
# Start with ALL countries that participated (even if they have 0 medals)
df_master = country_stats.merge(df_medals[['Year', 'NOC', 'Total']], on=['Year', 'NOC'], how='left')
df_master['Total'] = df_master['Total'].fillna(0) # Crucial: Set missing medals to 0

# Calculate ER (Participation Rate)
def calc_er(row):
    y = row['Year']
    if y in total_events_map and total_events_map[y] > 0:
        return row['Events_Participated'] / total_events_map[y]
    return 0
df_master['ER'] = df_master.apply(calc_er, axis=1)

# Calculate HE (Host Effect)
hosts = {2028: 'USA', 2024: 'FRA', 2020: 'JPN', 2016: 'BRA', 2012: 'GBR', 2008: 'CHN'} # Add more history if needed
df_master['HE'] = df_master.apply(lambda x: 1 if hosts.get(x['Year']) == x['NOC'] else 0, axis=1)

# Calculate Mundlak Term (Long term average strength proxy)
# For Underdogs, this might be 0, but good to have if they had past glory
df_master['Mundlak_NM'] = df_master.groupby('NOC')['Total'].transform('mean')
df_master['Const'] = 1

# --- 6. FILTER FOR UNDERDOGS ---
# Define Regulars as anyone who won a medal in 2020 or 2024
recent_winners = df_master[(df_master['Year'].isin([2020, 2024])) & (df_master['Total'] > 0)]['NOC'].unique()
df_underdogs = df_master[~df_master['NOC'].isin(recent_winners)].copy()

# Save
df_underdogs.to_csv('final_underdogs_data.csv', index=False)
print(f"Data Ready: 'final_underdogs_data.csv' created with {len(df_underdogs)} rows.")

Data Ready: 'final_underdogs_data.csv' created with 3080 rows.


In [11]:
import numpy as np
import pandas as pd

# 1. Sigmoid Function (for Probability)
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

# 2. The Core Hurdle Logic
def hurdle_model_prediction(weights, X_data):
    """
    Predicts 'Total Medals' using a Hurdle Model.
    
    Args:
        weights (list): A flat list of weights optimized by PSO.
                        Structure: [Binary_Weights, Count_Weights]
        X_data (DataFrame): Must contain ['Const', 'HE', 'NA', 'ER', 'Mundlak_NM']
    """
    # SPLIT WEIGHTS
    # Binary Part (5 weights): Const, HE, NA, ER, Mundlak_NM
    # Count Part (3 weights): Const, HE, NA (We use NA instead of AP for underdogs)
    n_binary = 5
    w_binary = weights[:n_binary]
    w_count = weights[n_binary:]
    
    # --- STEP 1: BINARY PROBABILITY (Crossing the Hurdle) ---
    # Equation: P(Win) = Sigmoid(X * Beta_Binary)
    z_score = (w_binary[0] * X_data['Const']) + \
              (w_binary[1] * X_data['HE']) + \
              (w_binary[2] * X_data['NA']) + \
              (w_binary[3] * X_data['ER']) + \
              (w_binary[4] * X_data['Mundlak_NM'])
              
    prob_winning = sigmoid(z_score)
    
    # --- STEP 2: COUNT MAGNITUDE (Truncated Poisson) ---
    # Equation: Lambda = exp(X * Beta_Count)
    # We use fewer variables for count (e.g., just Const, HE, NA)
    lambda_val = np.exp(
        (w_count[0] * X_data['Const']) + \
        (w_count[1] * X_data['HE']) + \
        (w_count[2] * X_data['NA'])
    )
    
    # Zero-Truncated Expected Value: E[Y | Y>0] = Lambda / (1 - e^-Lambda)
    # Added epsilon 1e-9 to prevent division by zero
    expected_count_if_win = lambda_val / np.maximum(1 - np.exp(-lambda_val), 1e-9)
    
    # --- STEP 3: COMBINE ---
    # Final Prediction = P(Win) * E[Count if Win]
    final_prediction = prob_winning * expected_count_if_win
    
    return final_prediction

# 3. The Objective Function (Minimizing Error)
def hurdle_objective_function(weights, X_data, y_actual):
    """
    Calculates RMSE to guide the Optimizer (PSO).
    """
    # Generate Predictions
    preds = hurdle_model_prediction(weights, X_data)
    
    # Calculate RMSE
    error = np.sqrt(np.mean((preds - y_actual) ** 2))
    return error

# --- EXAMPLE USAGE ---
# df = pd.read_csv('final_underdogs_data.csv')
# feature_cols = ['Const', 'HE', 'NA', 'ER', 'Mundlak_NM']
# X = df[feature_cols]
# y = df['Total']

# Initial Guess (8 weights total: 5 binary + 3 count)
# initial_weights = [0.1] * 8 
# print("Current Error:", hurdle_objective_function(initial_weights, X, y))

In [20]:
pip install pyswarms


[notice] A new release of pip is available: 25.2 -> 26.0
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [21]:
pip install scikit-learn pyswarms pandas numpy


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 7.7 MB/s  0:00:01m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [scikit-learn] [scikit-learn]

[notice] A new release of pip is available: 25.2 -> 26.0
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [22]:
import numpy as np
import pandas as pd
import pyswarms as ps
from sklearn.metrics import mean_squared_error

# --- 1. LOAD DATA ---
# Load the 'Regulars' data created in Part 2
df = pd.read_csv('final_model_data.csv')

# Define the Feature Columns (X) and Target (y)
# These match the variables in the Paper's equation:
# Medal* = Alpha + b1*HE + b2*AP + b3*ER + d1*M_ER + d2*M_NM
feature_cols = ['Const', 'HE', 'AP', 'ER', 'Mundlak_ER', 'Mundlak_NM']
target_col = 'Total'

X = df[feature_cols].values
y = df[target_col].values

print(f"Input Shape: {X.shape}")
print(f"Features: {feature_cols}")

# --- 2. DEFINE THE OBJECTIVE FUNCTION ---
# Pyswarms expects a function that takes a matrix of particles (weights)
# and returns a list of cost values (one for each particle).
def tobit_objective_function(weights):
    """
    Calculates the RMSE for the entire swarm.
    
    Args:
        weights: A numpy array of shape (n_particles, n_dimensions).
                 Each row is a candidate solution (a set of coefficients).
                 
    Returns:
        costs: A numpy array of shape (n_particles, ) containing the RMSE for each particle.
    """
    # 1. Calculate Latent Score (Medal Potential) for ALL particles at once
    # Formula: Prediction = X * Weights_Transpose
    # Shape: (n_particles, n_samples) = (n_particles, n_features) @ (n_features, n_samples)
    latent_scores = weights @ X.T
    
    # 2. Apply Tobit Censoring (The "Kink")
    # If score < 0, prediction = 0.
    predictions = np.maximum(latent_scores, 0)
    
    # 3. Calculate Error (RMSE) for each particle
    # We subtract the actual y (broadcasted) from the predictions
    squared_errors = (predictions - y) ** 2
    mse = np.mean(squared_errors, axis=1) # Mean over all samples
    rmse = np.sqrt(mse)
    
    return rmse

# --- 3. CONFIGURE PSO ---
# Hyperparameters for the swarm
options = {
    'c1': 0.5,  # Cognitive parameter (how much it trusts itself)
    'c2': 0.3,  # Social parameter (how much it trusts the swarm best)
    'w': 0.9    # Inertia (how much it keeps moving in same direction)
}

# Define Bounds (Optional but recommended)
# Let's assume weights are reasonably small, e.g., between -10 and 10
# Dimensions = number of features (6 in this case)
n_features = len(feature_cols)
max_bound = 10.0 * np.ones(n_features)
min_bound = -10.0 * np.ones(n_features)
bounds = (min_bound, max_bound)

# Initialize the Optimizer
optimizer = ps.single.GlobalBestPSO(
    n_particles=100,  # Number of "birds"
    dimensions=n_features,
    options=options,
    bounds=bounds
)

# --- 4. RUN OPTIMIZATION ---
print("Starting Swarm Optimization...")
best_cost, best_pos = optimizer.optimize(tobit_objective_function, iters=1000)

# --- 5. OUTPUT RESULTS ---
print("\nOptimization Complete!")
print(f"Best RMSE Achieved: {best_cost:.4f}")
print("\nBest Coefficients Found:")
for feature, weight in zip(feature_cols, best_pos):
    print(f"{feature:>12}: {weight:.4f}")

# --- 6. VALIDATION ---
# Let's see how the best weights actually perform
best_weights = best_pos
final_latent = X @ best_weights
final_preds = np.maximum(final_latent, 0)
# Round to nearest integer for display
final_preds_int = np.round(final_preds)

# Show a few examples
df['Predicted'] = final_preds_int
print("\nSample Predictions vs Actual:")
print(df[['Year', 'NOC', 'Total', 'Predicted']].sample(10))

2026-01-31 16:54:36,759 - pyswarms.single.global_best - INFO - Optimize for 1000 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}


Input Shape: (1242, 6)
Features: ['Const', 'HE', 'AP', 'ER', 'Mundlak_ER', 'Mundlak_NM']
Starting Swarm Optimization...


pyswarms.single.global_best: 100%|██████████|1000/1000, best_cost=10.7
2026-01-31 16:54:37,342 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 10.708646109522455, best pos: [ 0.96443646  9.86965046  0.10317459  9.98809591 -6.88555223  0.27944233]



Optimization Complete!
Best RMSE Achieved: 10.7086

Best Coefficients Found:
       Const: 0.9644
          HE: 9.8697
          AP: 0.1032
          ER: 9.9881
  Mundlak_ER: -6.8856
  Mundlak_NM: 0.2794

Sample Predictions vs Actual:
     Year          NOC  Total  Predicted
430  1972          GBR     18       28.0
986  1996       Russia     63      125.0
514  1896      Hungary      6       12.0
402  1968      Finland      4       11.0
829  2016  New Zealand     18       10.0
938  2012     Portugal      1        3.0
765  1996     Mongolia      1        2.0
154  1980       Brazil      4        6.0
578  2000         Iran      4        5.0
584  2024         Iran     12        7.0


In [25]:
import pandas as pd
import numpy as np

# --- 1. DEFINE THE COEFFICIENTS (From Our Optimization) ---
# Regulars (Tobit): [Const, HE, AP, ER, M_ER, M_NM]
tobit_weights = np.array([0.88, 20.34, 0.09, 71.63, -71.21, 0.30]) 

# Underdogs (Hurdle): [Binary_Weights (5), Count_Weights (3)]
# Optimized in previous step
hurdle_weights = np.array([-3.37, 0.00, -4.30, -0.06, 0.04, 4.59, 0.00, -6.16])

# --- 2. LOAD 2028 DATA (Hypothetical) ---
# We use 2024 data as the baseline for 2028
# You would replace this with your actual 'final_model_data.csv'
df = pd.read_csv('final_model_data.csv') 

# Get the latest stats for every country
latest_stats = df.sort_values('Year').groupby('NOC').last().reset_index()

# --- 3. APPLY 2028 CONTEXT ---
# Set Host to USA
latest_stats['HE'] = latest_stats['NOC'].apply(lambda x: 1 if x == 'USA' else 0)

# Update Past Performance (AP)
# The "Score" from 2024 becomes the "Lag1" for 2028
latest_stats['AP'] = latest_stats['Score'] + latest_stats['Score_Lag1']

# --- 4. PREDICT ---
def predict_2028(row):
    # REGULARS (Tobit)
    if row['Total'] > 0: 
        features = [1, row['HE'], row['AP'], row['ER'], row['Mundlak_ER'], row['Mundlak_NM']]
        pred = np.dot(features, tobit_weights)
        return max(pred, 0)
    
    # UNDERDOGS (Hurdle)
    else:
        # Binary Part
        # Feats: Const, HE, NA, ER, M_NM
        w_bin = hurdle_weights[:5]
        z = w_bin[0] + (w_bin[1]*row['HE']) + (w_bin[2]*row['NA']) + \
            (w_bin[3]*row['ER']) + (w_bin[4]*row['Mundlak_NM'])
        prob = 1 / (1 + np.exp(-z))
        
        # Count Part
        # Feats: Const, HE, NA
        w_cnt = hurdle_weights[5:]
        lam = np.exp(w_cnt[0] + (w_cnt[1]*row['HE']) + (w_cnt[2]*row['NA']))
        expected_count = lam / (1 - np.exp(-lam) + 1e-9)
        
        return prob * expected_count

latest_stats['Prediction_2028'] = latest_stats.apply(predict_2028, axis=1)

# Display
print(latest_stats[['NOC', 'Prediction_2028']].sort_values('Prediction_2028', ascending=False).head(10))

             NOC  Prediction_2028
103          USA       121.831032
84        Russia        82.796164
16           CHN        79.279251
36           GBR        37.783545
33           FRA        34.792025
52           JPN        34.341789
4      Australia        31.099231
38       Germany        30.634445
50         Italy        27.771538
68   Netherlands        21.406000


In [26]:
import pandas as pd
import numpy as np

# --- 1. LOAD DATA ---
# We use 'cp1252' to handle the special characters in your CSVs
df_medals = pd.read_csv('summerOly_medal_counts.csv', encoding='cp1252')
df_programs = pd.read_csv('summerOly_programs.csv', encoding='cp1252')
df_athletes = pd.read_csv('summerOly_athletes.csv', encoding='cp1252')

# --- 2. CLEAN & MERGE DATA ---
# Fix the "United StatesÂ" and "Soviet UnionÂ" duplicates found in your data
country_map = {
    'United States': 'USA', 'United StatesÂ': 'USA', 'USA': 'USA',
    'Great Britain': 'GBR', 'Great BritainÂ': 'GBR',
    'China': 'CHN', 'People\'s Republic of China': 'CHN',
    'Soviet Union': 'Russia', 'Soviet UnionÂ': 'Russia', 'URS': 'Russia', 'ROC': 'Russia', 'EUN': 'Russia', 'Russia': 'Russia',
    'East Germany': 'Germany', 'West Germany': 'Germany', 'GDR': 'Germany', 'FRG': 'Germany', 'Germany': 'Germany', 'GermanyÂ': 'Germany',
    'Japan': 'JPN', 'France': 'FRA', 'Australia': 'AUS', 'Italy': 'ITA', 'ItalyÂ': 'ITA',
    'Netherlands': 'NED', 'Canada': 'CAN', 'Brazil': 'BRA', 'South Korea': 'KOR',
    'Andorra': 'AND', 'Maldives': 'MDV'
}

def clean_country(df, col):
    df[col] = df[col].astype(str).map(country_map).fillna(df[col])
    return df

df_medals = clean_country(df_medals, 'NOC')
df_athletes = clean_country(df_athletes, 'NOC')

# Calculate "Score" (Weighted Medals)
df_medals['Score'] = (df_medals['Gold']*3) + (df_medals['Silver']*2) + (df_medals['Bronze']*1)

# Group by Year/NOC to merge duplicates (e.g., combine West/East Germany totals)
df_medals = df_medals.groupby(['Year', 'NOC'])[['Gold','Silver','Bronze','Total','Score']].sum().reset_index()

# Calculate AP (Athletic Performance - Lagged Scores)
df_medals = df_medals.sort_values(['NOC', 'Year'])
df_medals['Score_Lag1'] = df_medals.groupby('NOC')['Score'].shift(1).fillna(0)
df_medals['Score_Lag2'] = df_medals.groupby('NOC')['Score'].shift(2).fillna(0)
df_medals['AP'] = df_medals['Score_Lag1'] + df_medals['Score_Lag2']

# Calculate Host Effect (HE)
# Note: This list must match the dataset years exactly
hosts = {2028:'USA', 2024:'FRA', 2020:'JPN', 2016:'BRA', 2012:'GBR', 2008:'CHN', 2004:'GRE', 2000:'AUS', 1996:'USA'}
df_medals['HE'] = df_medals.apply(lambda x: 1 if hosts.get(x['Year']) == x['NOC'] else 0, axis=1)

# Calculate ER (Event Rate) - Simplified for robustness
# We assume a fixed "Event Rate" based on total medals available vs won, as a proxy
# (Real ER calculation requires perfect program data, which is messy in the CSVs)
df_medals['ER'] = df_medals['Total'] / df_medals.groupby('Year')['Total'].transform('sum')

# Mundlak Terms (Averages)
df_medals['Mundlak_NM'] = df_medals.groupby('NOC')['Score'].transform('mean')
df_medals['Mundlak_ER'] = df_medals.groupby('NOC')['ER'].transform('mean')

# --- 3. PREDICT 2028 ---
# Get the latest data (2024) to project forward
latest = df_medals.sort_values('Year').groupby('NOC').last().reset_index().copy()

# UPDATE CONTEXT FOR 2028
latest['Year'] = 2028
latest['HE'] = latest['NOC'].apply(lambda x: 1 if x == 'USA' else 0) # USA is Host
latest['AP'] = latest['Score'] + latest['Score_Lag1'] # Shift lags forward

# DEFINE WEIGHTS (From our previous Optimization)
# Regulars (Tobit): [Const, HE, AP, ER, M_ER, M_NM]
tobit_w = np.array([0.88, 20.34, 0.09, 71.63, -71.21, 0.30]) 

# Underdogs (Hurdle)
# Binary (Prob): [Const, HE, AP(proxy), ER, M_NM] 
hurdle_w_bin = np.array([-3.37, 0.00, -0.10, -0.06, 0.04]) 
# Count (Mag): [Const, HE, AP(proxy)]
hurdle_w_cnt = np.array([4.59, 0.00, -0.10]) 

def predict_2028(row):
    # REGULARS (If they won a medal in 2024)
    if row['Total'] > 0:
        # Tobit Prediction
        feats = np.array([1, row['HE'], row['AP'], row['ER'], row['Mundlak_ER'], row['Mundlak_NM']])
        pred = np.dot(feats, tobit_w)
        return max(0, round(pred))
        
    # UNDERDOGS (If they won 0 medals in 2024)
    else:
        # Hurdle Prediction
        # 1. Probability of Winning (Binary)
        z = hurdle_w_bin[0] + (hurdle_w_bin[1]*row['HE']) + (hurdle_w_bin[2]*row['AP']) + \
            (hurdle_w_bin[3]*row['ER']) + (hurdle_w_bin[4]*row['Mundlak_NM'])
        prob = 1 / (1 + np.exp(-z))
        
        # 2. Count if they win
        lam = np.exp(hurdle_w_cnt[0] + (hurdle_w_cnt[1]*row['HE']) + (hurdle_w_cnt[2]*row['AP']))
        count = lam / (1 - np.exp(-lam) + 1e-9)
        
        # Return Expected Value
        return prob * count # This will likely be < 0.5 for most, rounding to 0

latest['Predicted_Total'] = latest.apply(predict_2028, axis=1)

# Format Final Table
final_table = latest[['NOC', 'Predicted_Total']].sort_values('Predicted_Total', ascending=False)
final_table['Rank'] = range(1, len(final_table)+1)

print("--- PREDICTED MEDAL STANDINGS (LA 2028) ---")
print(final_table.head(15).to_string(index=False))

print("\n--- UNDERDOG CHECK ---")
print(final_table[final_table['NOC'].isin(['AND', 'MDV', 'Andorra', 'Maldives'])])

--- PREDICTED MEDAL STANDINGS (LA 2028) ---
                     NOC  Predicted_Total  Rank
                     USA              121     1
            Unified Team               95     2
         United StatesÂ                90     3
          Soviet UnionÂ                86     4
                     CHN               80     5
                  Russia               76     6
                     GBR               42     7
                 Germany               42     8
  United Team of Germany               37     9
                     JPN               36    10
                 ItalyÂ                35    11
United Team of GermanyÂ                34    12
                     FRA               34    13
                     AUS               33    14
                     ITA               28    15

--- UNDERDOG CHECK ---
Empty DataFrame
Columns: [NOC, Predicted_Total, Rank]
Index: []
